<a href="https://colab.research.google.com/github/Delonix7/My_Data_Science_Projects/blob/main/SQL_queries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Querying Chicago City Database

In [ ]:
# These libraries are pre-installed in SN Labs. If running in another environment please uncomment lines below to install them:
!pip install --force-reinstall ibm_db==3.1.0 ibm_db_sa==0.3.3
# Ensure we don't load_ext with sqlalchemy>=1.4 (incompadible)
!pip uninstall sqlalchemy==1.4 -y && pip install sqlalchemy==1.3.24
!pip install ipython-sql


In [ ]:
%load_ext sql

In [ ]:
import ibm_db

In [ ]:
%sql ibm_db_sa://qjx88370:7rHuWwPexRZAGL2p@b1bc1829-6f45-4cd4-bef4-10cf081900bf.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32304/bludb?Security=SSL

In [ ]:
%%sql
select tabname, tabschema, create_time from syscat.tables
where tabname
 in ('CENSUS_DATA', 'CHICAGO_CRIME_DATA', 'CHICAGO_PUBLIC_SCHOOLS')

In [ ]:
%%sql 
select name, coltype, length from sysibm.syscolumns where TBNAME = 'CENSUS_DATA'
order by name;

In [ ]:
%%sql
select name, coltype, length from sysibm.syscolumns where TBNAME = 'CHICAGO_CRIME_DATA'
order by name;

In [ ]:
%%sql
select name, coltype, length from sysibm.syscolumns where TBNAME = 'CHICAGO_PUBLIC_SCHOOLS'
order by name;

### Problem 1

##### Find the total number of crimes recorded in the CRIME table.


In [ ]:
%%sql
select count(case_number) as Total_Number_of_Crimes from CHICAGO_CRIME_DATA;

 * ibm_db_sa://qjx88370:***@b1bc1829-6f45-4cd4-bef4-10cf081900bf.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32304/bludb
Done.


total_number_of_crimes
533


### Problem 2

##### List community areas with per capita income less than 11000.

In [ ]:
%%sql
select COMMUNITY_AREA_NAME from CENSUS_DATA
where PER_CAPITA_INCOME < 11000;

 * ibm_db_sa://qjx88370:***@b1bc1829-6f45-4cd4-bef4-10cf081900bf.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32304/bludb
Done.


community_area_name
West Garfield Park
South Lawndale
Fuller Park
Riverdale


### Problem 3

##### List all case numbers for crimes  involving minors?(children are not considered minors for the purposes of crime analysis)


In [ ]:
%%sql
select CASE_NUMBER from CHICAGO_CRIME_DATA
where DESCRIPTION like '%MINOR%';

 * ibm_db_sa://qjx88370:***@b1bc1829-6f45-4cd4-bef4-10cf081900bf.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32304/bludb
Done.


case_number
HL266884
HK238408


### Problem 4

##### List all kidnapping crimes involving a child?


In [ ]:
%%sql
select case_number from CHICAGO_CRIME_DATA
where DESCRIPTION like '%CHILD %' and PRIMARY_TYPE like '%KIDN%';

 * ibm_db_sa://qjx88370:***@b1bc1829-6f45-4cd4-bef4-10cf081900bf.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32304/bludb
Done.


case_number
HN144152


### Problem 5

##### What kinds of crimes were recorded at schools?


In [ ]:
%%sql
select distinct(PRIMARY_TYPE) as Kinds_of_crime_reported_at_schools 
from CHICAGO_CRIME_DATA
where LOCATION_DESCRIPTION like '%SCHOOL%';

 * ibm_db_sa://qjx88370:***@b1bc1829-6f45-4cd4-bef4-10cf081900bf.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32304/bludb
Done.


kinds_of_crime_reported_at_schools
ASSAULT
BATTERY
CRIMINAL DAMAGE
CRIMINAL TRESPASS
NARCOTICS
PUBLIC PEACE VIOLATION


### Problem 6

##### List the average safety score for each type of school.


Renamed "Elementary, Middle, or High School" was changed to "SCHOOL_TYPE" for easy handling of data

In [ ]:
%%sql
Alter table "CHICAGO_PUBLIC_SCHOOLS"
rename column "Elementary, Middle, or High School" to "SCHOOL_TYPE" ;

In [ ]:
%%sql
select SCHOOL_TYPE, avg(SAFETY_SCORE) as AVERAGE_SAFETY_SCORE
from CHICAGO_PUBLIC_SCHOOLS 
group by SCHOOL_TYPE;

 * ibm_db_sa://qjx88370:***@b1bc1829-6f45-4cd4-bef4-10cf081900bf.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32304/bludb
Done.


school_type,average_safety_score
ES,49
HS,49
MS,48


### Problem 7

##### List 5 community areas with highest % of households below poverty line


In [ ]:
%%sql
select COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY
from CENSUS_DATA
order by PERCENT_HOUSEHOLDS_BELOW_POVERTY desc 
fetch first 5 rows only;

 * ibm_db_sa://qjx88370:***@b1bc1829-6f45-4cd4-bef4-10cf081900bf.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32304/bludb
Done.


community_area_name,percent_households_below_poverty
Riverdale,56.5
Fuller Park,51.2
Englewood,46.6
North Lawndale,43.1
East Garfield Park,42.4


### Problem 8

##### Which community area is most crime prone?


Renamed "COMMUNITY_AREA_NUMBER" in CENSUS_DATA to "COMM_AREA_NUM" to 
prevent conflict with the same column name in CHICAGO_CRIME_DATA

In [ ]:
%%sql
alter table CENSUS_DATA
rename column "COMMUNITY_AREA_NUMBER" to "COMM_AREA_NUM";

In [ ]:
%%sql 
select count(COMMUNITY_AREA_NUMBER) as Number_of_crimes, COMMUNITY_AREA_NUMBER
from CHICAGO_CRIME_DATA , CENSUS_DATA 
where COMMUNITY_AREA_NUMBER = COMM_AREA_NUM
group by COMMUNITY_AREA_NUMBER
order by Number_of_crimes desc
fetch first 1 row only;

 * ibm_db_sa://qjx88370:***@b1bc1829-6f45-4cd4-bef4-10cf081900bf.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32304/bludb
Done.


number_of_crimes,community_area_number
43,25


### Problem 9

##### Use a sub-query to find the name of the community area with highest hardship index


In [ ]:
%%sql
select COMMUNITY_AREA_NAME from CENSUS_DATA
where hardship_index = (select max(hardship_index) from CENSUS_DATA)

 * ibm_db_sa://qjx88370:***@b1bc1829-6f45-4cd4-bef4-10cf081900bf.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32304/bludb
Done.


community_area_name
Riverdale


### Problem 10

##### Use a sub-query to determine the Community Area Name with most number of crimes?


In [ ]:
%%sql
select COMMUNITY_AREA_NAME from CENSUS_DATA
where COMM_AREA_NUM  =(select community_area_number 
from(select count(COMMUNITY_AREA_NUMBER) 
as Number_of_crimes, COMMUNITY_AREA_NUMBER
from CHICAGO_CRIME_DATA
group by COMMUNITY_AREA_NUMBER
order by Number_of_crimes desc 
fetch first 1 row only))

 * ibm_db_sa://qjx88370:***@b1bc1829-6f45-4cd4-bef4-10cf081900bf.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32304/bludb
Done.


community_area_name
Austin


In [ ]:
%%sql 
select community_area_number 
from(select count(COMMUNITY_AREA_NUMBER) as Num, COMMUNITY_AREA_NUMBER
from CHICAGO_CRIME_DATA
group by COMMUNITY_AREA_NUMBER
order by Num desc 
fetch first 1 row only);

 * ibm_db_sa://qjx88370:***@b1bc1829-6f45-4cd4-bef4-10cf081900bf.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32304/bludb
Done.


community_area_number
25


In [ ]:
%%sql
select count(COMMUNITY_AREA_NUMBER) as Num, COMMUNITY_AREA_NUMBER
from CHICAGO_CRIME_DATA
group by COMMUNITY_AREA_NUMBER
order by Num desc 
fetch first 1 row only;


 * ibm_db_sa://qjx88370:***@b1bc1829-6f45-4cd4-bef4-10cf081900bf.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32304/bludb
Done.


num,community_area_number
43,25
